In [1]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 2 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:20.04

MAINTAINER Amazon AI <sage-learner@amazon.com>

ARG PYTHON_VERSION_TAG=3.8.3
ARG LINK_PYTHON_TO_PYTHON3=1

RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*
    
RUN apt-get -qq -y update && \
    DEBIAN_FRONTEND=noninteractive apt-get -qq -y install \
        gcc \
        g++ \
        zlibc \
        zlib1g-dev \
        libssl-dev \
        libbz2-dev \
        libsqlite3-dev \
        libncurses5-dev \
        libgdbm-dev \
        libgdbm-compat-dev \
        liblzma-dev \
        libreadline-dev \
        uuid-dev \
        libffi-dev \
        tk-dev \
        curl \
        git \
        make \
        sudo \
        bash-completion \
        tree \
        vim \
       

# Docker Image

In [1]:
%cd container
!docker build -t port-cnn .
%cd ../

/home/ec2-user/SageMaker/protein-annotation/inference-container/container
Sending build context to Docker daemon  377.9kB
Step 1/17 : FROM ubuntu:20.04
 ---> f643c72bc252
Step 2/17 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> 326b3d86eee1
Step 3/17 : ARG PYTHON_VERSION_TAG=3.8.3
 ---> Using cache
 ---> 439310e2c915
Step 4/17 : ARG LINK_PYTHON_TO_PYTHON3=1
 ---> Using cache
 ---> dbb5d06e3d11
Step 5/17 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 5f43ad46e6dd
Step 6/17 : RUN apt-get -qq -y update &&     DEBIAN_FRONTEND=noninteractive apt-get -qq -y install         gcc         g++         zlibc         zlib1g-dev         libssl-dev         libbz2-dev         libsqlite3-dev         libncurses5-dev         libgdbm-dev         libgdbm-compat-dev         liblzma-dev         libreadline-dev         uuid-dev         libffi-dev    

In [2]:
# Push image to ECR
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

ecr_repository = 'port-cnn'
tag = ':latest'
uri_suffix = 'amazonaws.com'
port_cnn_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

# Create ECR repository and push docker image
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $port_cnn_uri
!docker push $port_cnn_uri

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:877465308896:repository/port-cnn-15",
        "registryId": "877465308896",
        "repositoryName": "port-cnn-15",
        "repositoryUri": "877465308896.dkr.ecr.us-east-1.amazonaws.com/port-cnn-15",
        "createdAt": 1607546024.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}
The push refers to repository [877465308896.dkr.ecr.us-east-1.amazonaws.com/port-cnn-15]

c53ef381: Preparing 
c0f5558d: Preparing 
ec89fa3b: Preparing 
a32e937

# Build Endpoint

## Initializations

In [3]:
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [4]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [20]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/port-cnn:latest'.format(account, region)

In [23]:
sage = boto3.Session().client(service_name='sagemaker')

## Create Model

In [ ]:
model_name = "PortCNN-prediction"

In [24]:
# Add the location of model artifacts to ModelDataUrl field
primary_container = {
    'Image': image,
    'ModelDataUrl': "",
}

create_model_response = sage.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

## Create Endpoint Configuration

In [25]:
job_name_prefix = 'PortCNN-inference'

In [26]:
import time
from time import gmtime, strftime

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_config_name = job_name_prefix + '-epc-' + timestamp
endpoint_config_response = sage.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m5.4xlarge',
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print('Endpoint configuration name: {}'.format(endpoint_config_name))
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

Endpoint configuration name: PortCNN-inference-epc--2020-12-09-21-22-50
Endpoint configuration arn:  arn:aws:sagemaker:us-east-1:877465308896:endpoint-config/portcnn-inference-epc--2020-12-09-21-22-50


## Create Endpoint

In [27]:
sagemaker = boto3.client(service_name='sagemaker')

In [29]:
%%time
import time

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_name = job_name_prefix + '-ep-' + timestamp
print('Endpoint name: {}'.format(endpoint_name))

endpoint_params = {
    'EndpointName': endpoint_name,
    'EndpointConfigName': endpoint_config_name,
}
endpoint_response = sagemaker.create_endpoint(**endpoint_params)
print('EndpointArn = {}'.format(endpoint_response['EndpointArn']))

Endpoint name: PortCNN-inference-ep--2020-12-09-21-22-55
EndpointArn = arn:aws:sagemaker:us-east-1:877465308896:endpoint/portcnn-inference-ep--2020-12-09-21-22-55
CPU times: user 7.76 ms, sys: 4.05 ms, total: 11.8 ms
Wall time: 218 ms


In [30]:
# get the status of the endpoint
response = sagemaker.describe_endpoint(EndpointName=endpoint_name)
status = response['EndpointStatus']
print('EndpointStatus = {}'.format(status))


# wait until the status has changed
sagemaker.get_waiter('endpoint_in_service').wait(EndpointName=endpoint_name)


# print the status of the endpoint
endpoint_response = sagemaker.describe_endpoint(EndpointName=endpoint_name)
status = endpoint_response['EndpointStatus']
print('Endpoint creation ended with EndpointStatus = {}'.format(status))

if status != 'InService':
    raise Exception('Endpoint creation failed.')

EndpointStatus = Creating
Endpoint creation ended with EndpointStatus = InService
